In [241]:
#%pip install altair vega vega_datasets pandas scikit-learn sentence-transformers streamlit

## Visualization Project: Movie Recommendation System

#### Dataset: The Movies Dataset
This system uses the movies dataset (https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset). It has metadata on over 45,000 movies. 26 million ratings from over 270,000 users.

#### Objective
The objective of this project is to build a movie recommendation system using the metadata of the movies dataset. The system will recommend movies with the appropriate visualizations based on the user's preferences.

In [242]:
import pandas as pd
import altair as alt
from ast import literal_eval
movies_metadata = pd.read_csv('./The_Movies_Dataset/movies_metadata.csv', low_memory=False)

movies_metadata_genres = movies_metadata[['id', 'title', 'genres']].copy()

# convert the genres column to a list
movies_metadata_genres['genres'] = (
    movies_metadata_genres['genres']
    .fillna('[]')
    .apply(literal_eval)
    .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
)

In [243]:
# Grouping by genres
genres = movies_metadata_genres.explode('genres')
genres = genres.groupby('genres').size().reset_index(name='count')
genres = genres.sort_values('count', ascending=False)

# calculate the percentage
genres['percentage'] = genres['count'] / genres['count'].sum() * 100

In [244]:
import pandas as pd
import altair as alt
from ast import literal_eval
movies_metadata = pd.read_csv('./The_Movies_Dataset/movies_metadata.csv', low_memory=False)

In [245]:
# plot the genres distribution
alt.Chart(genres).mark_arc().encode(
    theta=alt.Theta(field='percentage', type='quantitative', stack=True),
    color=alt.Color(field='genres', type='nominal'),
    tooltip=[alt.Tooltip('genres', title='Genre'), alt.Tooltip('percentage', format='.2f', title='Percentage')]
).properties(
    title='Genres Distribution'
).interactive()

alt.Chart(...)

In [246]:
# Grouping by genres
genres = movies_metadata_genres.explode('genres')
genres = genres.groupby('genres').size().reset_index(name='count')
genres = genres.sort_values('count', ascending=False)
# calculate the percentage
genres['percentage'] = genres['count'] / genres['count'].sum() * 100
genres['top_movies'] = genres['genres'].apply(lambda x: ' '.join(f"{i+1}. {title}" for i, title in enumerate(movies_metadata_genres[movies_metadata_genres['genres'].apply(lambda y: x in y)]['title'].astype(str).head(10))))

In [247]:
# plot the genres distribution and show top 10 movies as a tooltip
alt.Chart(genres).mark_arc().encode(
    theta=alt.Theta(field='percentage', type='quantitative', stack=True),
    color=alt.Color(field='genres', type='nominal'),
    tooltip=[alt.Tooltip('genres', title='Genre'), alt.Tooltip('percentage', format='.2f', title='Percentage'), alt.Tooltip('top_movies', title='Top Movies')],
    order=alt.Order('count', sort='descending')
).properties(
    title='Genres Distribution and Top Movies'
).interactive()

alt.Chart(...)

In [248]:
credits = pd.read_csv('./The_Movies_Dataset/credits.csv', low_memory=False)
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')
credits['id'] = pd.to_numeric(credits['id'], errors='coerce')
movies_metadata_credits = movies_metadata.merge(credits, on='id')
# Calculating the average rating for each movie is not enough. We need to calculate the weighted rating for each movie.
# https://math.stackexchange.com/questions/169032/understanding-the-imdb-weighted-rating-function-for-usage-on-my-own-website
# Weighted Rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C

# v = number of votes for the movie
# m = minimum votes required to be listed in the chart
# R = average rating of the movie
# C = mean vote across the whole report

C = movies_metadata_credits['vote_average'].mean()
m = movies_metadata_credits['vote_count'].quantile(0.90)

# filter out movies that have less than m votes
movies_metadata_credits = movies_metadata_credits[movies_metadata_credits['vote_count'] >= m]

# calculate the score
movies_metadata_credits['score'] = (
    (movies_metadata_credits['vote_count'] / (movies_metadata_credits['vote_count'] + m)) * movies_metadata_credits['vote_average'] +
    (m / (movies_metadata_credits['vote_count'] + m)) * C
)
movies_metadata_credits = movies_metadata_credits.sort_values('score', ascending=False)
# print(movies_metadata_credits.head(10))

In [249]:
import os
from sentence_transformers import SentenceTransformer

model_name = 'all-MiniLM-L6-v2'
model_directory = './models'

# Create the models directory if it doesn't exist
os.makedirs(model_directory, exist_ok=True)

model_path = os.path.join(model_directory, model_name)

# Check if the model is already saved on disk
if os.path.exists(model_path):
    # Load the model from disk
    model = SentenceTransformer(model_path)
else:
    # Download the model and save it to disk
    model = SentenceTransformer(model_name)
    model.save(model_path)

In [250]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

movies_metadata_overview = movies_metadata[['id', 'title', 'overview', 'popularity', 'vote_average']].copy()
# cache the embeddings in a folder
embeddings_directory = './embeddings'
os.makedirs(embeddings_directory, exist_ok=True)
if os.path.exists(os.path.join(embeddings_directory, 'movies_metadata_overview_embeddings.npy')):
    embeddings = np.load(os.path.join(embeddings_directory, 'movies_metadata_overview_embeddings.npy'))
else:
    # Generate the embeddings
    embeddings = model.encode(movies_metadata_overview['overview'].astype(str).tolist())
    # Save the embeddings to disk
    np.save(os.path.join(embeddings_directory, 'movies_metadata_overview_embeddings.npy'), embeddings)

In [251]:
# Function to find similar movies
def find_similar_movies(movie_title, top_n=10):
    idx = movies_metadata_overview[movies_metadata_overview['title'] == movie_title].index[0]
    query_embedding = embeddings[idx].reshape(1, -1)

    # Calculate cosine similarity between the query and all movie embeddings
    similarities = cosine_similarity(query_embedding, embeddings).flatten()

    # Get the top N similar movies
    similar_indices = similarities.argsort()[::-1][1:top_n+1]
    similar_movies = movies_metadata_overview.iloc[similar_indices]

    return similar_movies

# pick 5 movies from the top 100 movies
filtered_movies = movies_metadata_credits.head(100).sample(5)
similar_movies = pd.concat([find_similar_movies(movie_title).assign(query_title=movie_title) for movie_title in filtered_movies['title']])
similar_movies = similar_movies.copy()
similar_movies['popularity'] = similar_movies['popularity'].astype(float).round(2)

dropdown_symbol = alt.binding_select(options=list(similar_movies["query_title"].unique()), name=" ")
selection_symbol = alt.selection_point(fields=["query_title"], bind=dropdown_symbol, value=filtered_movies['title'].values[0])

# display the similar movies on a chart
alt.Chart(similar_movies).mark_point().encode(
    alt.X('popularity', title='Popularity'),
    alt.Y('vote_average', title='Vote Average'),
    alt.Color('title', title='Title'),
    tooltip=['title', 'overview']
).add_params(
    selection_symbol
).transform_filter(
    selection_symbol
).properties(
    title='Similar Movies to Selected Movie',
    width=600,
    height=400,
).interactive()

alt.Chart(...)

In [252]:
movies_metadata_genres = movies_metadata[['id', 'title', 'genres', 'release_date']].copy()

# convert the genres column to a list
movies_metadata_genres['genres'] = (
    movies_metadata_genres['genres']
    .fillna('[]')
    .apply(literal_eval)
    .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
)

movies_metadata_genres['release_date'] = pd.to_datetime(movies_metadata_genres['release_date'], errors='coerce')
movies_metadata_genres = movies_metadata_genres[~movies_metadata_genres['release_date'].isna()]
# convert year to decade
movies_metadata_genres['year'] = (movies_metadata_genres['release_date'].dt.year // 10 * 10).astype(int)

genres = movies_metadata_genres.explode('genres')
genres = genres.groupby(['genres', 'year']).size().reset_index(name='count')
genres = genres.sort_values('count', ascending=False)

# Create a selection object for the slider
year_slider = alt.binding_range(min=1900, max=2010, step=10, name='Year:')
year_select = alt.selection_point(fields=['year'], bind=year_slider, value={'year': 1900})

# calculate the percentage
genres['percentage'] = genres['count'] / genres['count'].sum() * 100
# plot the genres distribution
chart = alt.Chart(genres).mark_bar().encode(
    x=alt.X('genres', title='Genre', sort='-y'),
    y=alt.Y('count', title='Count'),
    color=alt.Color('genres', title='Genre'),
    tooltip=[alt.Tooltip('genres', title='Genre'), alt.Tooltip('count', format='.2f', title='Count')]
).add_params(
    year_select
).transform_filter(
    year_select
).properties(
    title='Genres Distribution',
    width=600,
    height=400,
).interactive()

chart

alt.Chart(...)

In [253]:
import time
import json
from ast import literal_eval
import requests

# extract the country from the production_countries column
movies_metadata_countries = movies_metadata[['id', 'title', 'production_countries']].copy()

# convert the production_countries column to a list
movies_metadata_countries['production_countries'] = (
    movies_metadata_countries['production_countries']
    .fillna('[]')
    .apply(literal_eval)
    .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
)

# explode the production_countries column
movies_metadata_countries = movies_metadata_countries.explode('production_countries')
movies_metadata_countries = movies_metadata_countries.groupby('production_countries').size().reset_index(name='count')
movies_metadata_countries = movies_metadata_countries.sort_values('count', ascending=False)

def fetch_lat_long_data(country):
    country_param = country.replace(' ', '%20')
    country_filename = country.replace(' ', '_').lower() + '.json'
    data_directory = './data'
    if os.path.exists(os.path.join(data_directory, country_filename)):
        with open(os.path.join(data_directory, country_filename), 'r') as f:
            return json.load(f)
    else:
        lat_long_url = f"https://geocode.maps.co/search?q={country_param}&api_key=6668f5eb3d464108527128ndi77d74c"
        print('Fetching data for:', country)
        response = requests.get(lat_long_url)
        if response.status_code == 200:
            data = response.json()
            with open(os.path.join(data_directory, country_filename), 'w') as f:
                json.dump(data, f)
            print('Waiting for 5 seconds...')
            time.sleep(5)
            return data
        else:
            return None

def get_lat_long(country):
    data = fetch_lat_long_data(country)
    if data:
        return data[0]['lat'] + ',' + data[0]['lon']
    else:
        return None

movies_metadata_countries['lat_long'] = movies_metadata_countries.apply(lambda x: get_lat_long(x['production_countries']), axis=1)
movies_metadata_countries[['lat', 'long']] = movies_metadata_countries['lat_long'].str.split(',', expand=True)
movies_metadata_countries = movies_metadata_countries.drop(columns=['lat_long'])
# write the data to a csv file
movies_metadata_countries.to_csv('./movies_metadata_countries.csv', index=False)

In [281]:
import altair as alt
from vega_datasets import data

# Load a GeoJSON source that contains features for drawing
base = alt.topo_feature(data.world_110m.url, 'countries')

# Create a base map
base_map = alt.Chart(base).mark_geoshape(
    fill='lightblue',
    stroke='white'
).project('equirectangular')

# Create a scatter plot layer
points = alt.Chart(movies_metadata_countries).mark_circle().encode(
    longitude='long:Q',
    latitude='lat:Q',
    size=alt.Size('count:Q', title='Count', scale=alt.Scale(range=[50, 250])),
    # color=alt.Color('count:Q', title='Count'),
    color=alt.Color('count:Q', title='Count', scale=alt.Scale(range=['blue', 'darkblue'])),
    tooltip=[alt.Tooltip('production_countries', title='Country'), alt.Tooltip('count', format='.2f', title='Count')]
).project('equirectangular')

# Combine the base map and scatter plot
chart = base_map + points

# Set the chart properties
chart = chart.properties(
    title='Countries Distribution',
    width=800,
    height=600
).interactive()

chart

alt.LayerChart(...)